In [17]:
import pandas as pd
import numpy as np
import random
import re
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
file_path = 'C:/Users/poung/OneDrive/바탕 화면/capstone/질병.csv'
dis = pd.read_csv(file_path)

file_path2 = 'C:/Users/poung/OneDrive/바탕 화면/capstone/최최종식품DB.csv'
df = pd.read_csv(file_path2)
df.fillna(0, inplace=True)
file_path3 = 'C:/Users/poung/OneDrive/바탕 화면/capstone/modified_merged_dat.csv'
med = pd.read_csv(file_path3)
file_path4 = 'C:/Users/poung/OneDrive/바탕 화면/capstone/정렬데이터.csv'
sor = pd.read_csv(file_path4)

In [5]:
missing_columns = df.columns[df.isnull().any()].tolist()
print(missing_columns)

[]


In [6]:
import json
import pandas as pd
import re

#json 파일 읽어오기
with open( 'C:/Users/poungd/Downloads/img_json_file.json', encoding='utf-8') as file:
    datas = json.load(file)
#json 파일 접근 코드 -> json 파일에 inferText에 접근하여 영양정보 추출
field_data= datas['images'][0]['fields']
text_list = None
def field_infertext(field_data):
    global text_list
    text_list = []
    for i in range(len(field_data)):
        if ('탄수화무'in field_data[i]['inferText'] or '탄수화믈'in field_data[i]['inferText']or '탄수화몰'in field_data[i]['inferText']or'탄수와믈'in field_data[i]['inferText']or'탄수와몰'in field_data[i]['inferText']or'탄수와물'in field_data[i]['inferText']):
            field_data[i]['inferText'] = field_data[i]['inferText'].replace(field_data[i]['inferText'],'탄수화물')
        elif ('종 내용당'in field_data[i]['inferText'] or '내용량'in field_data[i]['inferText']):
            field_data[i]['inferText'] = field_data[i]['inferText'].replace(field_data[i]['inferText'],'총 내용량')
        if ('1일'in field_data[i]['inferText'] or '1일 영양성분'in field_data[i]['inferText']or '1일영양성분'in field_data[i]['inferText']):
            break
        text_list.append(field_data[i]['inferText'])
    return text_list
infer_sentence=field_infertext(field_data)


# 데이터 전처리
#name은 추가될 수 있음
name = "나트륨|탄수화물|당류|지방|트랜스지방|포화지방|콜레스테롤|단백질|비타민B1|비타민B2|비타민B6|칼슘"
result = {}
#딕셔너리 형태로 받아옴 ex) {나트륨:610mg,31%, 탄수화물:54g,17%, ....}
current_item = None
current_values = []
for item in infer_sentence:
    m_item=re.match(name, item)
    if m_item is not None:
        if current_item is not None:
            result[current_item] = current_values
        key_string=m_item.group()
        value_string=item.replace(key_string, "")
        current_item = key_string 
        if len(value_string) != 0: 
            current_values = [value_string] #리스트 초기화
        else:
            current_values = []
    else:
        current_values.append(item)#name

# 마지막 요소 추가
if current_item is not None:
    result[current_item] = current_values

#위에 받아온 정보들을 전처리 하고 데이터 프레임으로 받아올 수 있도록 정제
# g전처리 + 정규표현식 + value 에러 전처리----------------------------
for key,val in result.items():
    for i,v in enumerate(val):
        if len(val) != 0:
            result[key][i] = re.findall(r'\d+|[a-zA-Z]+|%+', v)
    result[key] = sum(val,[])
        

my_list = []  
lst_2 = []
key_name = list(result.keys())[0]
#ipdb.set_trace()
for key_2, val_2 in result.items():
    for idx, v_2 in enumerate(val_2):
        if len(my_list) == 0:
            my_list.append(v_2)
        elif re.search(r'[0-9a-zA-Z]|%', v_2):  
            if re.search(r'g|mg|%$', my_list[-1]):  
                join_str = ''.join(my_list)
                if re.match(r'^0\d.', join_str):
                    join_str = re.sub(r'^0', '0.', join_str)
                lst_2.append(join_str)
                my_list = [] #리셋
            my_list.append(v_2)
        if len(lst_2) != 0 and idx == 0:
            if len(lst_2) == 1 and lst_2 =='g':  
                lst_2 = '0g'   #'0g'인데 'g'으로만 인식이 되어 '0g'으로 전처리
            elif len(lst_2) == 1: 
                lst_2.append('Nan')
            elif len(lst_2) == 0:
                lst_2.append('Nan','Nan')
            result[key_name] = lst_2
            key_name = key_2
            lst_2 = [] #리셋

if len(my_list) != 0:
    join_str = ''.join(my_list)
    lst_2.append(join_str)
    result[key_name] = lst_2


#데이터 프레임으로 받아오기         
img_df = pd.DataFrame(result)
person = Person(age=25, gender='female', disease=['간경화'])
con_df(img_df,person)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/poungd/Downloads/img_json_file.json'

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## 대표적인 변경사항
 person객체에 먹은 음식들 영양정보를 저장하는 리스트가 생김
 약품 함수들 대거 추가
 영양 성분이 초과해도 그냥 영양분을 추

In [22]:
class Person:
    disease = []
    today_kcal = []
    today_pro = []
    today_car = []
    today_fat = []
    def cal_nut(self): #연령별로 칼로리계산
        if self.age <= 18:
            self.Kcal = 2700
        elif self.age <= 29:
            self.Kcal = 2600
        elif self.age <= 49:
            self.Kcal = 2400
        elif self.age <= 64:
            self.Kcal = 2200
        elif self.age <= 74:
            self.Kcal = 2300
        else:
            self.Kcal = 2000

        if self.gender == 'female':
            self.Kcal -= 400
    #생성자 함수
    def __init__(self, age, gender, carbo=324, sug=100, fats=54, tran=100, satf=14, chol=300, prot=55, calc=700, sodi=2000, today_kcal=None, today_pro=None, today_car=None, today_fat=None, disease=None):
        self.age = age
        self.gender = gender
        self.Kcal = 0
        self.carbo = int(carbo)
        self.sugars = int(sug)
        self.fats = int(fats)
        self.trans_fats = int(tran)
        self.saturated_fats = int(satf)
        self.cholesterol = int(chol)
        self.protein = int(prot)
        self.calcium = int(calc)
        self.sodium = int(sodi)
        self.disease = disease
        self.cal_nut()
        self.error()
        self.today_kcal = today_kcal if today_kcal is not None else []
        self.today_pro = today_pro if today_pro is not None else []
        self.today_car = today_car if today_car is not None else []
        self.today_fat = today_fat if today_fat is not None else []

    #일일영양성분 초과시 에러안내문
    def error(self):
        negative_values = {}
        if self.carbo < 0:
            negative_values['carbo'] = self.carbo
        if self.sugars < 0:
            negative_values['sugars'] = self.sugars
        if self.fats < 0:
            negative_values['fats'] = self.fats
        if self.trans_fats < 0:
            negative_values['trans_fats'] = self.trans_fats
        if self.saturated_fats < 0:
            negative_values['saturated_fats'] = self.saturated_fats
        if self.cholesterol < 0:
            negative_values['cholesterol'] = self.cholesterol
        if self.protein < 0:
            negative_values['protein'] = self.protein
        if self.calcium < 0:
            negative_values['calcium'] = self.calcium
        if self.sodium < 0:
            negative_values['sodium'] = self.sodium
        
        if negative_values:
            for key, value in negative_values.items():
                print("일일 영양성분 초과입니다 ")
                print(f"{key}")
    #매일 매일 섭취정보 초기화
    def daily_reset(self):
        self.cal_nut()
        self.carbo = self.c
        self.sugars = self.s
        self.fats = self.f
        self.trans_fats = self.t
        self.saturated_fats = self.sa
        self.cholesterol = self.ch
        self.protein = self.p
        self.calcium = self.ca
        self.sodium = self.so
        self.today_pro = []
        self.today_car = []
        self.today_fat = []

    #여유분의 에너지 반환
    def extra_nut(self):
        return (self.Kcal, self.carbo, self.sugars, self.fats, 
                self.trans_fats, self.saturated_fats, self.cholesterol, 
                self.protein, self.calcium, self.sodium)

    #음식 섭취시 불러오는 함수
    def eat(self, kal=0, car=0, sug=0, fat=0, trans=0, sat=0, chol=0, pro=0, cal=0, sod=0):
        self.Kcal -= kal
        self.carbo -= car
        self.sugars -= sug
        self.fats -= fat
        self.trans_fats -= trans
        self.saturated_fats -= sat
        self.cholesterol -= chol
        self.protein -= pro
        self.calcium -= cal
        self.sodium -= sod
        self.append_eat(kal,pro,car,fat) #음식 섭취시에 자동으로 추가
        
    def custom(self, **key):
        for nut, val in key.items():
            if hasattr(self, nut):
                setattr(self, nut, val)

    #질병 반환
    def disease_return(self):
        return self.disease
    #질병 추가
    def append_disease(self,disease):
        self.disease.append(disease)
    #오늘 섭취한음식 리스트에 추가 그림그리기용
    def append_eat(self, kcal, pro, car, fat): #오늘 섭취한 음식 추가
        if len(self.today_kcal) > 0:
            kcal += self.today_kcal[-1] 
        self.today_kcal.append(kcal)
        self.today_pro.append(pro)
        self.today_car.append(car)
        self.today_fat.append(fat)
    #오늘 섭취한 음식 리스트 반환
    def return_eat(self):
        return self.today_kcal, self.today_pro, self.today_car, self.today_fat
#이름으로 음식 검색 이름만 반환
def find_food(food_name, df):
    food_list = df[df['식품명'].str.contains(food_name, case=False)]
    
    if food_list.empty:
        print("해당하는 음식이 없습니다.")
    else:
        print("해당하는 음식 리스트:")
        for index, row in food_list.iterrows():
            print(row['식품명'])
        return food_list.index.tolist()

#이름으로 음식 검색 음식 번호가 나옴
def select_food(food_name, df):
    food_info = df[df['식품명'].str.contains(food_name, case=False)]
    
    if food_info.empty:
        print("해당하는 음식이 없습니다.")
    else:
        print("해당하는 음식 정보:")
        return(food_info.iloc[:, 1:])

#카테고리에 포함되는 음식만 반환
def select_food_category(category, df):
    food_info = df[df['분류'].str.contains(category, case=False)]
    
    if food_info.empty:
        print("해당하는 음식이 없습니다.")
    else:
        print("해당하는 음식 정보:")
        return food_info.iloc[:, 1:]
#음식 거부시 추천 알고리즘으로 넘기는 함수
def deny_food(food_num, df):
    recommend = recommend_food(food_num, df)
    print(recommend)
#음식 섭취시 호출하는 함수
def confirm_food(food_num, df, person):
    food_info = df.iloc[food_num, 2:12]
    person_nutrients = person.extra_nut()
    #섭취 가능여부 판별 추후에 계속 사용
    if any(person_nutrients[i] <= food_info.iloc[i] for i in range(len(person_nutrients))):
         print("일일 영양섭취 초과입니다")
         deny_food(food_num, df)
    else:
         person.eat(*food_info)


#음식 추천 알고리즘 similar함수를 이용해 이름의 유사도를 비교
def recommend_food(food_num, df):
    selected_food = df.iloc[food_num]
    category_name = selected_food['분류']
    kcal, carbo, sugar, fats, trans_fats, sat_fats, cholesterol, protein, calcium, sodium = person.extra_nut()
    
    sim_foods = df[(df['분류'] == category_name) &
                   (df['에너지(kcal)'] <= kcal) &
                   (df['탄수화물(g)'] <= carbo) &
                   (df['지방(g)'] <= fats) &
                   (df['당류(g)'] <= sugar) &
                   (df['트랜스지방산(g)'] <= trans_fats) &
                   (df['포화지방산(g)'] <= sat_fats) &
                   (df['콜레스테롤(mg)'] <= cholesterol) &
                   (df['단백질(g)'] <= protein) &
                   (df['칼슘(mg)'] <= calcium) &
                   (df['나트륨(mg)'] <= sodium)].copy()
    #정렬후 리턴
    sim_foods['similarity'] = sim_foods['식품명'].apply(lambda x: similar(x, selected_food['식품명']))
    sim_foods = sim_foods.sort_values(by='similarity', ascending=False)

    return sim_foods.iloc[0:10, 1:]
#이름 비교 시스템
def similar(a, b):
    if a == b:
        return 1.0
    else:
        n, m = len(a), len(b)
        if n > m:
            a, b = b, a
            n, m = m, n

        current_row = range(n+1)
        for i in range(1, m+1):
            previous_row, current_row = current_row, [i] + [0] * n
            for j in range(1, n+1):
                add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
                if a[j-1] != b[i-1]:
                    change += 1
                current_row[j] = min(add, delete, change)

        return 1 - (current_row[n] / max(m, n))
#랜덤으로 음식을 뽑아주는 시스템
def ran_food_recom(df):
    ind = random.sample(range(len(df)), 10)
    sel = df.iloc[ind, 1:]
    return sel
#음식 섭취 영양성분을 매개변수로 받음
def consume_food(per, kal=0, car=0, sug=0, fat=0, trans=0, sat=0, chol=0, pro=0, cal=0, sod=0):
    # 오늘의 여유분 계산
    kcal, carbo, sugars, fats, trans_fats,sat_fats, cholesterol, protein, calcium, sodium = per.extra_nut()  
    
    if all([
        kcal >= kal,
        carbo >= car,
        sugars >= sug,
        fats >= fat,
        trans_fats >= trans,
        sat_fats >= sat,
        cholesterol >= chol,
        protein >= pro,
        calcium >= cal,
        sodium >= sod
    ]):
        print("섭취 가능한 품목입니다.")
    else:
        print("일일 영양섭취 초과입니다.")
#섭취가능여부를 판별하지 않고 섭취
def consume_food2(per, kal=0, car=0, sug=0, fat=0, trans=0, sat=0, chol=0, pro=0, cal=0, sod=0): #무조건 먹는거
    per.eat(kal, car, sug, fat, trans, sat, chol, pro, cal, sod)
    print("입력한 데이터가 입력되었습니다.")

#이미지에서 이름을 추출하여 넣을시 리스트에서 가장 비슷한 음식 하나를 골라 섭취 가능여부 판별
def food_classification(food_name, df,person):
    
    person_nutrients = person.extra_nut()
    df['similarity'] = df['식품명'].apply(lambda x: similar(x,food_name))
    df = df.sort_values(by='similarity', ascending=False)
    food_info = df.iloc[0, 2:12]
    if any(person_nutrients[i] <= food_info.iloc[i] for i in range(len(person_nutrients))):
        print("섭취 가능한 식품입니다")
    else:
        print("일일 영양성분 초과입니다")
    print(df[0:])

#질병 보유자를 위한 함수 다른점은 df2라는 질병 데이터를 받음
def dis_confirm_food(food_num, df, df2, person):
    food_info = df.iloc[food_num, 2:12]
    dis = person.disease_return()
    person_nutrients = person.extra_nut()
    if any(person_nutrients[i] <= int(food_info.iloc[i]) for i in range(len(person_nutrients))):
        foods = []
        precautions = []  # 주의사항을 저장할 리스트를 초기화합니다.
        for disease_name in dis:#질병명에 있는 권장식품으로 필터링
            matching_row = df2[df2['질병명'] == disease_name]
            foods.extend(matching_row['권장식품'].tolist())  # 수정된 부분
            precautions.extend(matching_row['주의사항'].tolist())  # 주의사항을 가져와 리스트에 추가합니다.
        print("일일 영양섭취 초과입니다")
        recommended_foods = dis_recommend_food(food_num, df, foods)
        
        print("권장 음식:", recommended_foods)
        print("주의사항:", precautions)
        return recommend_food
    else:
        person.eat(*food_info)
        return True

#recommendfood와 동일 대신 정렬과 중복제거를 함
def dis_recommend_food(food_num, df, foods):
    selected_food = df.iloc[food_num]
    category_name = selected_food['분류']
    kcal, carbo, sugar, fats, trans_fats, sat_fats, cholesterol, protein, calcium, sodium = person.extra_nut()
    
    sim_foods = df[(df['분류'] == category_name) &
                   (df['에너지(kcal)'] <= kcal) &
                   (df['탄수화물(g)'] <= carbo) &
                   (df['지방(g)'] <= fats) &
                   (df['당류(g)'] <= sugar) &
                   (df['트랜스지방산(g)'] <= trans_fats) &
                   (df['포화지방산(g)'] <= sat_fats) &
                   (df['콜레스테롤(mg)'] <= cholesterol) &
                   (df['단백질(g)'] <= protein) &
                   (df['칼슘(mg)'] <= calcium) &
                   (df['나트륨(mg)'] <= sodium)].copy()

    sim_foods['similarity'] = sim_foods['식품명'].apply(lambda x: similar(x, selected_food['식품명']))
    sim_foods = sim_foods.sort_values(by='similarity', ascending=False)
    food_list = pd.DataFrame()
    food_reshape = [food.replace(" ", "") for food in foods]
    food_reshape = ''.join(food_reshape)
    food_reshape = food_reshape.split(",")

    food_list = pd.DataFrame()
    for food in food_reshape:
        filtered_food_list = sim_foods[sim_foods['식품명'].str.contains(re.escape(food), case=False)]
        food_list = pd.concat([food_list, filtered_food_list])
    food_list.drop_duplicates(subset=['식품명'], inplace=True)
    food_list = food_list.sort_values(by='similarity', ascending=False)
    food_list.drop(food_list[food_list['식품코드'] == food_num].index, inplace=True)
    return food_list[1:11]
#질병 추가 함수
def dis_append(person,dis_name):
    person.append_disease(dis_name)
#데이터를 너무 영양성분들이 높거나 낮은 데이터를 제거함
def remake(df, kcal=2600, carbo=324, fats=54, sugar=100, trans_fats=100, sat_fats=14, cholesterol=300, protein=55, calcium=700, sodium=2600):
    ma = 0.8 #최대 수치 비율
    mi = 0.2 #최소 수치 비율

    foods = df[(df['분류'] == "밥류") & 
               (kcal*mi <= df['에너지(kcal)']) & (df['에너지(kcal)'] <= kcal*ma) &
               (carbo*mi <= df['탄수화물(g)']) & (df['탄수화물(g)'] <= carbo*ma) &
               (fats*mi <= df['지방(g)']) & (df['지방(g)'] <= fats*ma) &
               (sugar*mi <= df['당류(g)']) & (df['당류(g)'] <= sugar*ma) &
               (trans_fats*mi <= df['트랜스지방산(g)']) & (df['트랜스지방산(g)'] <= trans_fats*ma) &
               (sat_fats*mi <= df['포화지방산(g)']) & (df['포화지방산(g)'] <= sat_fats*ma) &
               (cholesterol*mi <= df['콜레스테롤(mg)']) & (df['콜레스테롤(mg)'] <= cholesterol*ma) &
               (protein*mi <= df['단백질(g)']) & (df['단백질(g)'] <= protein*ma) &
               (calcium*mi <= df['칼슘(mg)']) & (df['칼슘(mg)'] <= calcium*ma) &
               (sodium*mi <= df['나트륨(mg)']) & (df['나트륨(mg)'] <= sodium*ma)].copy()
    return foods

def make_graph(person): #person넣으면 자동으로 그래프 그려줌
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.plot(person.today_kcal, label='칼로리', color='lime', marker='o', linestyle='-')
    plt.xlabel('식사 번호')
    plt.ylabel('칼로리 (kcal)')
    plt.title('오늘 섭취한 칼로리')
    plt.legend(loc='upper left')
    plt.ylim(0, 3000)
    for i in range(500, 3000, 500):
        plt.axhline(y=i, color='gray', linestyle='--', linewidth=0.5)

    labels = ['섭취량', '나머지']
    colors = ['#ff9999', '#66b3ff']
    explode = (0.1, 0)
    
    wedgeprops = {'width' : 0.75}
    
    plt.subplot(2, 2, 2)
    values = [sum(person.return_eat()[2]), 324 - sum(person.return_eat()[2])]
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors, explode=explode,shadow=True)
    plt.title('탄수화물 섭취량')

    plt.subplot(2, 2, 3)
    values = [sum(person.return_eat()[1]), 55 - sum(person.return_eat()[1])]
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors, explode=explode,shadow=True)
    plt.title('단백질 섭취량')
    
    plt.subplot(2, 2, 4)
    values = [sum(person.return_eat()[3]), 54 - sum(person.return_eat()[3])]
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors, explode=explode,shadow=True)
    plt.title('지방 섭취량')

    plt.tight_layout()
    plt.show()

#ocr에서 찍은 이미지를 변환하는것에 쓰임
def con_df(df, per):
    sodium = make_num(df.at[0, '나트륨'])
    carbohydrates = make_num(df.at[0, '탄수화물'])
    sugars = make_num(df.at[0, '당류'])
    fat = make_num(df.at[0, '지방'])
    trans_fat = make_num(df.at[0, '트랜스지방'])
    saturated_fat = make_num(df.at[0, '포화지방'])
    cholesterol = make_num(df.at[0, '콜레스테롤'])
    protein = make_num(df.at[0, '단백질'])
    calcium = make_num(df.at[0, '칼슘'])
    consume_food(per, kal=0, car=int(carbohydrates[0]), sug=int(sugars[0]), fat=int(fat[0]), trans=int(trans_fat[0]), sat=int(saturated_fat[0]), chol=int(cholesterol[0]), pro=int(protein[0]), cal=int(calcium[0]), sod=int(sodium[0]))
# 숫자 +글자에서 숫자만 출력함
def make_num(text):
    numbers = re.findall(r'\d+', text)
    return numbers

#df 3종류를 넣고 person을 받아서 추천
def sort_and_merge(food_list, sor, col_name, ascending=True, percentage=0.3):
    im_list = sor.sort_values(by=col_name, ascending=ascending)
    im_list = im_list.head(int(len(im_list) * percentage))
    merged_df = pd.merge(food_list, im_list, on='식품명', how='inner')
    return merged_df

def nat_rec(person, df, df2, sor, case): 
    person_nutrients = person.extra_nut()
    kcal, carbo, sugar, fats, trans_fats, sat_fats, cholesterol, protein, calcium, sodium = person.extra_nut()
    
    food_list = df[(df['에너지(kcal)'] <= kcal) &  # 데이터 검열
                   (df['탄수화물(g)'] <= carbo) &
                   (df['지방(g)'] <= fats) &
                   (df['당류(g)'] <= sugar) &
                   (df['트랜스지방산(g)'] <= trans_fats) &
                   (df['포화지방산(g)'] <= sat_fats) &
                   (df['콜레스테롤(mg)'] <= cholesterol) &
                   (df['단백질(g)'] <= protein) &
                   (df['칼슘(mg)'] <= calcium) &
                   (df['나트륨(mg)'] <= sodium)].copy()
    # 질병 보유자 검열
    if person.disease_return(): 
        dis = person.disease_return()
        foods = []
        for disease_name in dis:
            matching_row = df2[df2['질병명'] == disease_name]
            foods.extend(matching_row['권장식품'].tolist())

        food_list = pd.DataFrame()
        food_reshape = [food.replace(" ", "") for food in foods]
        food_reshape = ''.join(food_reshape)
        food_reshape = food_reshape.split(",")
    
        for food in food_reshape:
            filtered_food_list = df[df['식품명'].str.contains(re.escape(food), case=False)]
            food_list = pd.concat([food_list, filtered_food_list])
        food_list.drop_duplicates(subset=['식품명'], inplace=True)
    
    merged_df = food_list
    for num in case:
        if num == 1:  # 에너지 하위 30프로
            merged_df = sort_and_merge(merged_df, sor, '에너지(kcal)', percentage=0.3)
        elif num == 2:  # 에너지 상위 30프로
            merged_df = sort_and_merge(merged_df, sor, '에너지(kcal)', ascending=False, percentage=0.3)
        elif num == 3:  # 저지방
            merged_df = sort_and_merge(merged_df, sor, '지방(g)', percentage=0.7)
        elif num == 4:  # 고단백
            merged_df = sort_and_merge(merged_df, sor, '단백질(g)', ascending=False, percentage=0.9999)
        elif num == 5:  # 저나트륨
            merged_df = sort_and_merge(merged_df, sor, '나트륨(mg)', percentage=0.3)
        elif num == 6:  # 저탄수화물
            merged_df = sort_and_merge(merged_df, sor, '탄수화물(g)', percentage=0.3)
        elif num == 7:  # 저콜레스테롤
            merged_df = sort_and_merge(merged_df, sor, '콜레스테롤(mg)', percentage=0.3)
        elif num == 8:  # 저콜레스테롤
            merged_df = sort_and_merge(merged_df, sor, '에너지(kcal)', percentage=0.3)
        elif num == 9:  # 저콜레스테롤
            merged_df = sort_and_merge(merged_df, sor, '탄수화물(g)', percentage=0.3)
        elif num == 10:  # 저콜레스테롤
            merged_df = sort_and_merge(merged_df, sor, '당류(g)', percentage=0.3)
        elif num == 11:  # 저콜레스테롤
            merged_df = sort_and_merge(merged_df, sor, '지방(g)', percentage=0.3)
        elif num == 12:  # 트랜스지방산
            merged_df = sort_and_merge(merged_df, sor, '트랜스지방산(g)', percentage=0.3)
        elif num == 13:  # 포화지방산
            merged_df = sort_and_merge(merged_df, sor, '포화지방산(g)', percentage=0.3)
        elif num == 14:  # 콜레스테롤
            merged_df = sort_and_merge(merged_df, sor, '콜레스테롤(mg)', percentage=0.3)
        elif num == 15:  # 칼슘
            merged_df = sort_and_merge(merged_df, sor, '단백질(g)', percentage=0.3)
        elif num == 16:  # 나트륨
            merged_df = sort_and_merge(merged_df, sor, '칼슘(mg)', percentage=0.3)
        elif num == 17:  # 식이섬유
            merged_df = sort_and_merge(merged_df, sor, '나트륨(mg)', percentage=0.3)

    ind = random.sample(range(len(merged_df)), min(10, len(merged_df)))
    sel = merged_df.iloc[ind, 1:12]
    sel.fillna(0, inplace=True)
    return sel


def check_min(person):
    nut = person.extra_nut()
    negative = []

    for val in nut:
        if val < 0:
            if val == person.Kcal:
                negative.append(8)
            elif val == person.carbo:
                negative.append(8)
            elif val == person.sugars:
                negative.append(9)
            elif val == person.fats:
                negative.append(10)
            elif val == person.trans_fats:
                negative.append(11)
            elif val == person.saturated_fats:
                negative.append(12)
            elif val == person.cholesterol:
                negative.append(13)
            elif val == person.protein:
                negative.append(14)
            elif val == person.calcium:
                negative.append(15)
            elif val == person.sodium:
                negative.append(16)

    return negative

def comb_confirm_food(food_num,person, df, df2):
    if person.disease_return() != []:
        return dis_confirm_food(food_num,df, df2,person)
    else:
        return confirm_food(food_num,df, person)

In [26]:
isinstance(person.extra_nut()[1],int)

True

In [13]:
def find_med(med, name):
    return med[med['제품명'].str.contains(name)].copy()
def find_med_row(med, number):
    med2 = med.iloc[number].copy()
    med2 = med2.fillna("해당사항 없음")
    return med2
def image_med(med, number):
    med2 = med.iloc[number].copy()
    return med2['ITEM_IMAGE']

In [25]:
person = Person(age=25, gender='female', disease=['간경화'])
#현재 음식 섭취시 무조건 식단 추천으로 변경해놓은 상태
dis_confirm_food(15, df, dis, person)
dis_confirm_food(15, df, dis, person)
consume_food2(person, kal=600, car=0, sug=0, fat=0, trans=0, sat=0, chol=0, pro=0, cal=0, sod=0)
comb_confirm_food(15,person, df, dis)
#make_graph(person)
#food_classification("허쉬 초콜릿",df,person)
#kkk = nat_rec(person, df, dis, sor, [1,3,4,5])
#kkk["에너지(kcal)_x"]
#print(check_min(person))
#nat_rec(person, df, dis, sor, check_min(person))

입력한 데이터가 입력되었습니다.
일일 영양섭취 초과입니다
권장 음식:                       식품코드              식품명  에너지(kcal)  탄수화물(g)  당류(g)  지방(g)  \
73168  P123-201020300-2385      밥류_잘지은곤약현미밥         86    18.18   0.00   0.82   
81214  P123-212020300-0441    반찬_지례흑돼지매콤불고기         18     2.13   0.33   0.29   
83574  P123-218020300-0193          순대_고기순대          8     1.50   0.00   0.15   
81215  P123-212020300-0442   반찬_지례흑돼지콩나물불고기         14     1.64   0.14   0.25   
85269  P123-226020100-0095  샐러드_오늘의과일/사과오렌지         52    12.67  10.33   0.47   
85271  P123-226020100-0097  샐러드_오늘의과일/파인오렌지         50    13.00   9.67   0.00   
81273  P123-212020300-0500         반찬_큐브고구마        128    29.66   0.00   0.41   
73806  P123-201020300-3023         밥류_현미곤약밥         77    20.00   0.00   0.33   
220    D107-128180000-0001         고구마_찐고구마        139    32.47  16.32   0.24   
73279  P123-201020300-2496         밥류_찰진흑미밥        155    35.71   0.48   0.43   

       트랜스지방산(g)  포화지방산(g)  콜레스테롤(mg)  단백질(g)  칼슘(mg)  나트륨(mg)  분류  s

<function __main__.recommend_food(food_num, df)>

In [5]:
print(find_med_row(med,33))

품목기준코드                                                197700120
제품명                                          부루펜정200밀리그램(이부프로펜)
문항1           이 약은 류마티양 관절염, 연소성 류마티양 관절염, 골관절염(퇴행성 관절질환), 감...
문항2           성인은 류마티양 관절염, 골관절염, 강직성 척추염, 연조직손상, 비관절 류마티스질환...
문항3                                                     해당사항 없음
문항4           이 약에 과민증 환자, 위장관궤양, 위장관출혈, 심한 혈액이상, 심한 간장애, 심한...
문항5           항암요법으로 고용량 메토트렉세이트를 투여중인 환자, 다른 비스테로이드성 소염진통제와...
문항6           쇽 증상(호흡곤란, 혈압저하 등), 헤마토크리트감소, 헤모글로빈감소, 빈혈, 재생불...
ITEM_IMAGE    https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...
Name: 33, dtype: object
